In [1]:
!pip install openpyxl
!pip install pdfplumber
!pip install -U langchain-community
!pip install chromadb
!pip install transformers accelerate datasets peft trl bitsandbytes
!pip install langchain
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.1 MB/s eta 0:00:00


In [2]:
# Importar librerías
import requests
import pandas as pd
import datetime
import time
import openpyxl
from sentence_transformers import SentenceTransformer
import pdfplumber
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
import torch
from trl import SFTTrainer
import gc
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import os
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI



In [3]:
posibles_params = {
    "page": 0,  # Número de página (empieza en 0)
    "pageSize": 50,  # Tamaño de página
    "order": "numeroConvocatoria",  # Campo por el que ordenar
    "direccion": "asc",  # Sentido de la ordenación: 'asc' o 'desc'
    "vpd": "GE",  # Identificador del portal
    "descripcion": "Resolución",  # Texto a buscar en el título o descripción
    "descripcionTipoBusqueda": 0,  # 0: frase exacta, 1: todas las palabras, 2: alguna palabra
    "numeroConvocatoria": "376046",  # Código BDNS a buscar
    "mrr": False,  # Mecanismo de recuperación y resiliencia
    "fechaDesde": "18/12/2017",  # Fecha de inicio (dd/mm/yyyy)
    "fechaHasta": "18/12/2017",  # Fecha de fin (dd/mm/yyyy)
    "tipoAdministracion": "C",  # 'C', 'A', 'L', 'O'
    "organos": ["713", "4730"],  # Lista de identificadores de órganos administrativos
    "regiones": [3, 50],  # Lista de identificadores de regiones
    "tiposBeneficiario": [3],  # Lista de identificadores de tipos de beneficiarios
    "instrumentos": [1],  # Lista de identificadores de instrumentos de ayuda
    "finalidad": 11,  # Identificador de la finalidad de la política de gasto
    "ayudaEstado": "SA.45221"  # Código de ayuda de estado
}

In [4]:
# Configuración de parámetros
base_url = "https://www.pap.hacienda.gob.es/bdnstrans/api/convocatorias/busqueda"
vpd = "GE"  # Identificador del portal, según la docu
page_size = 25
max_paginas = 3  # Puedes aumentar si quieres más resultados

resultados = []


# 2. Probar la API con parámetros y cabecera Accept: application/json
params = {
    "vpd": vpd,
    "page": 0,
    "pageSize": page_size
}
headers = {"Accept": "application/json"}

# Realizar una primera solicitud para verificar el Content-Type
try:
    r2 = requests.get(base_url, params=params, headers=headers)
    r2.raise_for_status() # Lanza una excepción para errores HTTP (4xx o 5xx)
except requests.exceptions.RequestException as e:
    print(f"❌ Error al conectar con la API o respuesta inicial: {e}")
    exit() # Salir si la conexión inicial falla

# 3. Si la respuesta es JSON, continuar con la descarga paginada
if "application/json" in r2.headers.get("Content-Type", ""):
    print("✅ La API responde con JSON. Descargando datos paginados...")
    for pagina in range(0, max_paginas):
        print(f"📄 Cargando página {pagina}...")
        params["page"] = pagina
        try:
            response = requests.get(base_url, params=params, headers=headers)
            response.raise_for_status()
            data = response.json()
            convocatorias = data.get("convocatorias", data.get("content", []))  # content es común en APIs paginadas
            if not convocatorias:
                print("✅ No hay más datos.")
                break
            resultados.extend(convocatorias)
            time.sleep(0.5)  # para evitar sobrecargar la API
        except Exception as e:
            print(f"❌ Error en la página {pagina}: {e}")
            break
    # Convertir a DataFrame y mostrar
    df = pd.DataFrame(resultados)
    print("Columnas disponibles:", df.columns.tolist())
    # Mostrar las primeras columnas si existen
    cols = [c for c in ["id", "titulo", "organoConvocante", "fechaPublicacion"] if c in df.columns]
else:
    print("❌ La API no responde con JSON. Revisa los parámetros, la URL o si la API está disponible.")

✅ La API responde con JSON. Descargando datos paginados...
📄 Cargando página 0...
📄 Cargando página 1...
📄 Cargando página 2...
Columnas disponibles: ['id', 'mrr', 'numeroConvocatoria', 'descripcion', 'descripcionLeng', 'fechaRecepcion', 'nivel1', 'nivel2', 'nivel3', 'codigoInvente']


In [5]:
df.head(10)  # Mostrar las primeras filas del DataFrame

# Guardar en un archivo Excel
output_file = "listado_convocatorias.xlsx"
df.to_excel(output_file, index=False)

In [6]:
df.head(10)  # Mostrar las primeras filas del DataFrame

,id,mrr,numeroConvocatoria,descripcion,descripcionLeng,fechaRecepcion,nivel1,nivel2,nivel3,codigoInvente
0,1050681,False,849120,CONVOCATORIA DE SUBVENCIONES PARA EL SECTOR PR...,None,2025-07-29,SANTIAGO DEL TEIDE,AYUNTAMIENTO DE SANTIAGO DEL TEIDE,None,None
1,1050680,False,849119,"Decreto n.º 97/2025,19 junio, Consejería Fomen...",None,2025-07-29,REGIÓN DE MURCIA,CONSEJERÍA DE FOMENTO E INFRAESTRUCTURAS,None,None
2,1050679,False,849118,"Decreto n.º 97/2025,19 junio, Consejería Fomen...",None,2025-07-29,REGIÓN DE MURCIA,CONSEJERÍA DE FOMENTO E INFRAESTRUCTURAS,None,None
3,1050678,False,849117,"Decreto n.º 97/2025,19 junio, Consejería Fomen...",None,2025-07-29,REGIÓN DE MURCIA,CONSEJERÍA DE FOMENTO E INFRAESTRUCTURAS,None,None
4,1050677,False,849116,"Decreto n.º 97/2025,19 junio, Consejería Fomen...",None,2025-07-29,REGIÓN DE MURCIA,CONSEJERÍA DE FOMENTO E INFRAESTRUCTURAS,None,None
5,1050676,False,849115,"Decreto n.º 97/2025,19 junio, Consejería Fomen...",None,2025-07-29,REGIÓN DE MURCIA,CONSEJERÍA DE FOMENTO E INFRAESTRUCTURAS,None,None
6,1050675,False,849114,"Decreto n.º 97/2025,19 junio, Consejería Fomen...",None,2025-07-29,REGIÓN DE MURCIA,CONSEJERÍA DE FOMENTO E INFRAESTRUCTURAS,None,None
7,1050674,False,849113,"Decreto n.º 97/2025,19 junio, Consejería Fomen...",None,2025-07-29,REGIÓN DE MURCIA,CONSEJERÍA DE FOMENTO E INFRAESTRUCTURAS,None,None
8,1050673,False,849112,"Decreto n.º 97/2025,19 junio, Consejería Fomen...",None,2025-07-29,REGIÓN DE MURCIA,CONSEJERÍA DE FOMENTO E INFRAESTRUCTURAS,None,None
9,1050672,False,849111,"Decreto n.º 97/2025,19 junio, Consejería Fomen...",None,2025-07-29,REGIÓN DE MURCIA,CONSEJERÍA DE FOMENTO E INFRAESTRUCTURAS,None,None


In [7]:
base_url = "https://www.infosubvenciones.es/bdnstrans/api/convocatorias"
params = {
    "vpd": "GE",         # Cambia por el portal que te interese
    "numConv": "842695"   # Número de convocatoria
}
headers = {"Accept": "application/json"}

print("🔎 Consultando convocatoria por parámetros...")
r = requests.get(base_url, params=params, headers=headers)
print("Status code:", r.status_code)
print("URL final:", r.url)
print("Content-Type:", r.headers.get("Content-Type"))
print("="*60)

if "application/json" in r.headers.get("Content-Type", ""):
    data = r.json()
    # Si la respuesta es una lista, conviértela directamente
    if isinstance(data, list):
        convocatoria = pd.DataFrame(data)
    # Si es un dict, conviértelo en DataFrame de una fila
    elif isinstance(data, dict):
        convocatoria = pd.DataFrame([data])
    else:
        print("Respuesta inesperada:", data)
        convocatoria = pd.DataFrame()
    print("Columnas disponibles:",convocatoria.columns.tolist())
else:
    print("❌ La API no responde con JSON. Revisa los parámetros, la URL o si la API está disponible.")
    convocatoria = pd.DataFrame()

🔎 Consultando convocatoria por parámetros...
Status code: 200
URL final: https://www.infosubvenciones.es/bdnstrans/api/convocatorias?vpd=GE&numConv=842695
Content-Type: application/json
Columnas disponibles: ['id', 'organo', 'sedeElectronica', 'codigoBDNS', 'fechaRecepcion', 'instrumentos', 'tipoConvocatoria', 'presupuestoTotal', 'mrr', 'descripcion', 'descripcionLeng', 'tiposBeneficiarios', 'sectores', 'regiones', 'descripcionFinalidad', 'descripcionBasesReguladoras', 'urlBasesReguladoras', 'sePublicaDiarioOficial', 'abierto', 'fechaInicioSolicitud', 'fechaFinSolicitud', 'textInicio', 'textFin', 'ayudaEstado', 'urlAyudaEstado', 'fondos', 'reglamento', 'objetivos', 'sectoresProductos', 'documentos', 'anuncios', 'advertencia']


In [8]:
convocatoria

,id,organo,sedeElectronica,codigoBDNS,fechaRecepcion,instrumentos,tipoConvocatoria,presupuestoTotal,mrr,descripcion,...,textFin,ayudaEstado,urlAyudaEstado,fondos,reglamento,objetivos,sectoresProductos,documentos,anuncios,advertencia
0,1044256,"{'nivel1': 'ANDALUCÍA', 'nivel2': 'CONSEJERÍA ...",None,842695,2025-06-26,[{'descripcion': 'SUBVENCIÓN Y ENTREGA DINERAR...,Concesión directa - instrumental,3208705.97,False,"Real Decreto 905/2022, de 25 de octubre, por e...",...,None,None,None,[{'descripcion': 'FEAGA - FONDO EUROPEO AGRÍCO...,None,[],[],"[{'id': 1286483, 'descripcion': 'Texto en cast...",[],La reutilización de los datos del Sistema Naci...


In [9]:
# Ejemplo de respuesta de una convocatoria
convocatoria

# Guardar la convocatoria en un archivo Excel
convocatoria_file = "convocatoria_842695.xlsx"
convocatoria.to_excel(convocatoria_file, index=False)

In [10]:
# Supón que ya tienes el DataFrame 'convocatoria' y quieres descargar todos los documentos
docs = convocatoria.iloc[0]['documentos']  # Si solo hay una convocatoria

# Carpeta donde guardar los documentos
os.makedirs("documentos_convocatoria", exist_ok=True)

for doc in docs:
    id_doc = doc['id']
    nombre = doc.get('nombreFic', f"documento_{id_doc}.pdf")
    url = f"https://www.infosubvenciones.es/bdnstrans/api/convocatorias/documentos?idDocumento={id_doc}"
    print(f"Descargando {nombre} ...")
    resp = requests.get(url)
    if resp.status_code == 200:
        with open(os.path.join("documentos_convocatoria", nombre), "wb") as f:
            f.write(resp.content)
        print(f"✅ Guardado: {nombre}")
    else:
        print(f"❌ Error al descargar {nombre} (status {resp.status_code})")

Descargando TextoConvocatoria.pdf ...
✅ Guardado: TextoConvocatoria.pdf


In [11]:
pdf_folder = "documentos_convocatoria" # Asegúrate de que esta carpeta exista y contenga tus PDFs
output_txt_file = "TextoConvocatoria.txt" # El archivo de texto unificado

all_text = []

# Recorrer todos los archivos en la carpeta de PDFs
for filename in os.listdir(pdf_folder):
    if filename.endswith(".pdf"):
        filepath = os.path.join(pdf_folder, filename)
        print(f"Extrayendo texto de: {filename}")
        try:
            with pdfplumber.open(filepath) as pdf:
                for page in pdf.pages:
                    text = page.extract_text()
                    if text: # Asegurarse de que se extrajo algo de texto
                        all_text.append(text)
        except Exception as e:
            print(f"Error al procesar {filename}: {e}")

# Unir todo el texto extraído en una sola cadena
unified_text = "\n".join(all_text)

# Guardar el texto unificado en un archivo .txt
with open(output_txt_file, "w", encoding="utf-8") as f:
    f.write(unified_text)

print(f"\nTexto de {len(os.listdir(pdf_folder))} PDFs extraído y guardado en '{output_txt_file}'")


Extrayendo texto de: TextoConvocatoria.pdf

Texto de 1 PDFs extraído y guardado en 'TextoConvocatoria.txt'


In [28]:
# Dividir en chunks para RAG
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_text(unified_text)
print(f"Texto dividido en {len(texts)} chunks")

Texto dividido en 460 chunks


In [31]:
api_key = 'INTRODUCIR LA API KEY'

In [34]:
client = OpenAI(api_key=api_key)

In [35]:
user_query = "¿Qué ocurre con el Alojamiento, manutención y comidas colectivas?"

In [39]:
# 2. Crear embeddings
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=api_key)
vectorstore = FAISS.from_texts(texts, embedding_model)

In [40]:

# 3. Buscar contexto relevante
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
relevant_docs = retriever.get_relevant_documents("¿Qué ocurre con el Alojamiento, manutención y comidas colectivas?")

# 4. Concatenar el contexto
context = "\n\n".join([doc.page_content for doc in relevant_docs])
user_query = "¿Qué ocurre con el Alojamiento, manutención y comidas colectivas?"

/tmp/ipython-input-40-110954037.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents("¿Qué ocurre con el Alojamiento, manutención y comidas colectivas?")


In [41]:
# 5. Llamar a GPT-4o-mini con contexto manual
client = OpenAI(api_key=api_key)
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Eres un asistente experto en ayudas públicas en España. Responde en tono claro, basado únicamente en el contexto que se te proporciona."},
        {"role": "user", "content": f"Contexto:\n{context}\n\nPregunta:\n{user_query}"}
    ],
    temperature=0.3,
    max_tokens=100  # Limitamos la salida del modelo a 100 tokens para probar como funciona. En el futuro habrá que deslimitalo

)

print("\n🧠 Respuesta:")
print(response.choices[0].message.content)


🧠 Respuesta:
El Alojamiento, manutención y comidas colectivas son gastos subvencionables por la participación en eventos fuera del lugar de trabajo. Se establecen las siguientes condiciones:

1. **Alojamiento**: Se abonará una dieta máxima de 120 euros/persona/día en España y 180 euros/persona/día en terceros países, siempre que se presenten las facturas pagadas que incluyan las fechas de estancia, nombre de los huéspedes y su vinculación con el beneficiario


In [42]:
def responder_pregunta(query, k=3, max_tokens=100):
    # Buscar contexto relevante
    docs = vectorstore.similarity_search(query, k=k)
    context = "\n\n".join([doc.page_content for doc in docs])

    # Llamar a GPT-4o-mini con contexto
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Eres un asistente experto en ayudas públicas en España. Responde con claridad y cita las fuentes."},
            {"role": "user", "content": f"Contexto:\n{context}\n\nPregunta:\n{query}"}
        ],
        temperature=0.3,
        max_tokens=max_tokens
    )

    # Mostrar respuesta
    print("\n🧠 Respuesta:")
    print(response.choices[0].message.content)

    # Mostrar documentos fuente
    print("\n📄 Fuentes:")
    for doc in docs:
        print(f"- Fuente: {doc.metadata.get('source', 'Desconocida')}")
        print(f"  Fragmento:\n{doc.page_content[:250]}...\n")

In [43]:
while True:
    user_query = input("\n❓ Escribe tu pregunta (o 'salir'): ")
    if user_query.lower() in ["salir", "exit", "quit"]:
        break
    responder_pregunta(user_query)


❓ Escribe tu pregunta (o 'salir'): ¿Qué ocurre con el Alojamiento, manutención y comidas colectivas?

🧠 Respuesta:
El apartado sobre **Alojamiento, manutención y comidas colectivas** establece las condiciones y límites para la subvención de estos gastos en el contexto de la participación en eventos fuera del lugar de trabajo. A continuación, se resumen los puntos clave:

1. **Alojamiento**:
   - Se puede abonar una dieta máxima de **120 euros por persona y día** en España y **180 euros por persona y día** en terceros países.
   - Es necesario presentar facturas pagadas

📄 Fuentes:
- Fuente: Desconocida
  Fragmento:
c) Alojamiento, manutención y comidas colectivas. Estas dietas se abonarán por la
participación en eventos fuera del lugar de trabajo, y cubrirán el número de días
necesario para la realización de la sub-acción.
– Se abonará una dieta máxima por el a...

- Fuente: Desconocida
  Fragmento:
convenientemente desglosados en el presupuesto recapitulativo del programa que se pres